Note, this tutorial assumes both NeMo and nemo_tools are installed, see [`this`](https://github.com/NVIDIA/NeMo) and [`this`](https://github.com/NVIDIA/NeMo/tree/main/nemo_tools) for details.

In [ ]:
BRANCH = 'main'

import os
import pynini

# Task Description

Inverse text normalization (ITN), also called denormalization, is a part of the Automatic Speech Recognition (ASR) post-processing pipeline. 

ITN is the task of converting the raw spoken output of the ASR model into its written form to improve the text readability. For example, `in nineteen seventy` should be changed to `in 1975` and `one hundred and twenty three dollars` to `$123`.

# NeMo Inverse Text Normalization

The NeMo ITN tool is a Python package that is based on weighted finite-state
transducer (WFST) grammars. The tool uses [`Pynini`](https://github.com/kylebgorman/pynini) to construct WFSTs, and the created grammars can be exported and integrated into [`Sparrowhawk`](https://github.com/google/sparrowhawk) (an open source version of [The Kestrel TTS text normalization system](https://www.cambridge.org/core/journals/natural-language-engineering/article/abs/kestrel-tts-text-normalization-system/F0C18A3F596B75D83B75C479E23795DA)) for production. The NeMo ITN tool is a Python extension of `Sparrowhawk`. 

Currently, NeMo tool provides support for English language and the following semiotic classes from the [Google Text normalization dataset](https://www.kaggle.com/richardwilliamsproat/text-normalization-for-english-russian-and-polish):
DATE, CARDINAL, MEASURE, DECIMAL, ORDINAL, MONEY, PLAIN. 

The toolkit is modular, easily extendable, and can be adapted to other languages and tasks like text normalization. The Python environment enables easy combination of text covering grammars with NNs. 

The overall NeMo ITN pipeline from development in `Pynini` to deployment in `Sparrowhawk` is shown below:
![alt text](deployment.png "Inverse Text Normalization Pipeline")

# Quick Start

## Add ITN to your Python ASR post-processing workflow:

In [ ]:
from nemo_tools.text_denormalization.denormalize import denormalize

raw_text = "we paid one hundred and twenty three dollars for this desk, and this."
denormalize(raw_text, verbose=False)

In the above cell, `one hundred and twenty three dollars` would be converted to `$123`, and the rest of the words remain the same.

## Run Inverse Text Normalization on an input from a file:

In [ ]:
# update nemo_tools path

NEMO_TOOLS_PATH = '/home/ebakhturina/NeMo/nemo_tools/text_denormalization'
DATA_DIR = '/home/ebakhturina/DEL'
os.makedirs(DATA_DIR, exist_ok=True)

INPUT_FILE = f'{DATA_DIR}/test.txt'
OUTPUT_FILE = f'{DATA_DIR}/test_itn.txt'

! echo "on march second twenty twenty" > $DATA_DIR/test.txt
! python $NEMO_TOOLS_PATH/run_predict.py --input=$INPUT_FILE --output=$OUTPUT_FILE

In [ ]:
# check that the raw text was indeed converted to the written form

! cat $OUTPUT_FILE

## Run evaluation

[Google Text normalization dataset](https://www.kaggle.com/richardwilliamsproat/text-normalization-for-english-russian-and-polish) consists of 1.1 billion words of English text from Wikipedia, divided across 100 files. The normalized text is obtained with [The Kestrel TTS text normalization system](https://www.cambridge.org/core/journals/natural-language-engineering/article/abs/kestrel-tts-text-normalization-system/F0C18A3F596B75D83B75C479E23795DA)).

Although a large fraction of this dataset can be reused for ITN by swapping input with output, the dataset is not bijective. 

For example: `1,000 -> one thousand`, `1000 -> one thousand`, `3:00pm -> three p m`, `3 pm -> three p m` are valid data samples for normalization but the inverse does not hold for ITN. 

We used regex rules to disambiguate samples where possible, see `nemo_tools/text_denormalization/clean_eval_data.py`.

To run evaluation, the `input` file should follow the Google Text normalization dataset format.

Example evaluation run: python run_evaluate.py --input=./en_with_types/output-00001-of-00100 [--denormalizer nemo]

In [ ]:
eval_text = """PLAIN	ON	<self>
DATE	22 July 2012	the twenty second of july twenty twelve
PUNCT	.	sil
<eos>	<eos>
"""

INPUT_FILE_EVAL = f'{DATA_DIR}/test_eval.txt'

with open(INPUT_FILE_EVAL, 'w') as f:
    f.write(eval_text)
    
! python $NEMO_TOOLS_PATH/run_evaluate.py --input=$INPUT_FILE_EVAL

In [ ]:
! cat $INPUT_FILE_EVAL

# C++ deployment

The instructions on how to export `Pynini` grammars and to run them with `Sparrowhawk`, could be found at [NeMo/text_denormalization/tools/text_denormalization](https://github.com/NVIDIA/NeMo/tree/text_denormalization/tools/text_denormalization).

# WFST and Common Pynini Operations

Finite-state acceptor (or FSA) is finite state autoaotom that has a finite number of states and no output. FSA either accepts (when the matching patter if found) or rejects a string (no match is found). 

In [ ]:
print([byte for byte in bytes('fst', 'utf-8')])

# create an acceptor from a string
pynini.accep('fst')

Here `0` - is a start note, `1` and `2` are the accept nodes, while `3` is a finite state.
By default (token_type="byte", `Pynini` interprets the string as a sequence of bytes, assigning one byte per arc. 

A finite state transducer (FST) not only matches the pattern but also produces output according to the defined transition states.

In [ ]:
# create an FST
pynini.cross('fst', 'FST')

Pynini supports the following operations:

- `closure` - Computes concatenative closure.
- `compose` - Constructively composes two FSTs.
- `concat` - Computes the concatenation (product) of two FSTs.
- `difference` - Constructively computes the difference of two FSTs.
- `invert`  - Inverts the FST's transduction.
- `optimize` - Performs a generic optimization of the FST.
- `project` - Converts the FST to an acceptor using input or output labels.
- `shortestpath` - Construct an FST containing the shortest path(s) in the input FST.
- `union`- Computes the union (sum) of two or more FSTs.


The list of most commonly used `Pynini` operations could be found [https://github.com/kylebgorman/pynini/blob/master/CHEATSHEET](https://github.com/kylebgorman/pynini/blob/master/CHEATSHEET). 

Pynini examples could be found at [https://github.com/kylebgorman/pynini/tree/master/pynini/examples](https://github.com/kylebgorman/pynini/tree/master/pynini/examples).
Use `help()` to explore the functuonality. For example:

In [ ]:
help(pynini.union)

# NeMo ITN API

NeMo ITN defines the following APIs that are called in sequence:

- `classify()` - creates a linear automaton from the input string and composes it with the final classification WFST, which transduces numbers and inserts semantic tags.  
- `parse()` - parses the tagged string into a list of key-value items representing the different semiotic tokens.
- `generate_reorderings()` - takes the parsed tokens and generates string serializations with different reorderings of the key-value items. This is important since WFSTs can only process input linearly, but the word order can change from spoken to written form (e.g., `three dollars -> $3`). 
- `verbalize()` - takes the intermediate string representation and composes it with the final verbalization WFST, which removes the tags and returns the written form.  

![alt text](pipeline.png "Inverse Text Normalization Pipeline")

# References and Furhter Reading:

- [Ebden, Peter, and Richard Sproat. "The Kestrel TTS text normalization system." Natural Language Engineering 21.3 (2015): 333.](https://www.cambridge.org/core/journals/natural-language-engineering/article/abs/kestrel-tts-text-normalization-system/F0C18A3F596B75D83B75C479E23795DA)
- [Gorman, Kyle. "Pynini: A Python library for weighted finite-state grammar compilation." Proceedings of the SIGFSM Workshop on Statistical NLP and Weighted Automata. 2016.](https://www.aclweb.org/anthology/W16-2409.pdf)
- [Mohri, Mehryar, Fernando Pereira, and Michael Riley. "Weighted finite-state transducers in speech recognition." Computer Speech & Language 16.1 (2002): 69-88.](https://cs.nyu.edu/~mohri/postscript/csl01.pdf)